In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from model import (QuadraticNN1, QuadraticNN2)

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Hyper-parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [5]:
model = QuadraticNN2(input_size, hidden_size, num_classes)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

QuadraticNN(
  (fc1): QuadraticNeurons(in_features=784, out_features=500)
  (relu): ReLU()
  (fc2): QuadraticNeurons(in_features=500, out_features=10)
)


In [6]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 7.9254
Epoch [1/5], Step [200/600], Loss: 4.1474
Epoch [1/5], Step [300/600], Loss: 3.3124
Epoch [1/5], Step [400/600], Loss: 0.4252
Epoch [1/5], Step [500/600], Loss: 0.8328
Epoch [1/5], Step [600/600], Loss: 1.3506
Epoch [2/5], Step [100/600], Loss: 0.4058
Epoch [2/5], Step [200/600], Loss: 0.6539
Epoch [2/5], Step [300/600], Loss: 0.5484
Epoch [2/5], Step [400/600], Loss: 0.3063
Epoch [2/5], Step [500/600], Loss: 0.8173
Epoch [2/5], Step [600/600], Loss: 0.2326
Epoch [3/5], Step [100/600], Loss: 0.1134
Epoch [3/5], Step [200/600], Loss: 0.1365
Epoch [3/5], Step [300/600], Loss: 0.1301
Epoch [3/5], Step [400/600], Loss: 0.3692
Epoch [3/5], Step [500/600], Loss: 0.3086
Epoch [3/5], Step [600/600], Loss: 0.1390
Epoch [4/5], Step [100/600], Loss: 0.1394
Epoch [4/5], Step [200/600], Loss: 0.0535
Epoch [4/5], Step [300/600], Loss: 0.0858
Epoch [4/5], Step [400/600], Loss: 0.6308
Epoch [4/5], Step [500/600], Loss: 0.0456
Epoch [4/5], Step [600/600], Loss:

In [7]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 93.58 %
